In [1]:
pip install opencv-python pillow numpy tk
# Note
# To turn off the live video press the live video button again


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import cv2
import numpy as np
import threading


class YOLOFaceDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("YOLO Object Detection and Filters")

        # Set the window size
        self.root.geometry("800x600")

        # Create a style object
        self.style = ttk.Style()

        # Define a style for buttons
        self.style.configure(
            "TButton",
            background="#00bcd4",  # Cyan background
            foreground="#000000",  # Black text
            font=("Arial", 12, "bold"),
            padding=10,
        )
        self.style.map(
            "TButton",
            background=[("active", "#0097a7")],
            foreground=[("active", "#ffffff")],
        )

        # Define a style for frames
        self.style.configure(
            "TFrame", background="#00bcd4"
        )  # Cyan background for frames

        # Create a panel for displaying images
        self.panel = ttk.Label(root)
        self.panel.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

        # Create a notebook (tabs) for organizing content
        self.notebook = ttk.Notebook(root)
        self.notebook.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Apply background color to the main window
        self.root.configure(bg="#00bcd4")

        # Create tabs for detection and filters
        self.create_detection_tab()
        self.create_filter_tab()

        # Initialize variables
        self.image_path = None
        self.video_path = None
        self.image = None
        self.video_capture = None
        self.net = None
        self.classes = None
        self.output_layers = None
        self.filter_mode = None
        self.detect_objects_flag = False
        self.running = False
        self.thread = None

    def create_detection_tab(self):
        detection_frame = ttk.Frame(self.notebook, padding="10", style="TFrame")
        self.notebook.add(detection_frame, text="Detection")

        # Use grid layout for better alignment
        detection_frame.grid_rowconfigure(0, weight=1)
        detection_frame.grid_columnconfigure(0, weight=1)

        # Create a canvas and a scrollbar
        canvas = tk.Canvas(detection_frame, bg="#00bcd4")
        scrollbar = ttk.Scrollbar(
            detection_frame, orient="vertical", command=canvas.yview
        )
        scrollbar.grid(row=0, column=1, sticky="ns")
        canvas.grid(row=0, column=0, sticky="nsew")
        canvas.configure(yscrollcommand=scrollbar.set)

        # Create a frame to contain the filter buttons
        filter_frame = ttk.Frame(canvas, padding="10", style="TFrame")
        canvas.create_window((0, 0), window=filter_frame, anchor="nw")

        # Bind configure event to update the scroll region
        filter_frame.bind(
            "<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )

        buttons = [
            ("Select Weights", self.select_weights),
            ("Select CFG", self.select_cfg),
            ("Select Names", self.select_names),
            ("Select Image", self.select_image),
            ("Select Video", self.select_video),
            ("Live Video", self.toggle_live_video),
            ("Detect Objects", self.toggle_detect_objects),
        ]

        # Add buttons to the filter_frame
        for idx, (text, command) in enumerate(buttons):
            btn = ttk.Button(filter_frame, text=text, command=command)
            btn.grid(row=idx, column=0, pady=5, sticky="ew")

    def create_filter_tab(self):
        filter_tab = ttk.Frame(self.notebook, padding="10", style="TFrame")
        self.notebook.add(filter_tab, text="Filters (8 different)")

        # Create a canvas for scrolling
        canvas = tk.Canvas(filter_tab, bg="#00bcd4")
        canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # Create a scrollbar for the canvas
        scrollbar = ttk.Scrollbar(filter_tab, orient="vertical", command=canvas.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        # Create a frame to contain the filter buttons
        filter_frame = ttk.Frame(canvas, padding="10", style="TFrame")
        filter_frame.bind(
            "<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        canvas.create_window((0, 0), window=filter_frame, anchor="nw")

        # Add filters as buttons
        filters = [
            ("Edge Detection", self.toggle_edge_detection),
            ("Sharpen", self.toggle_sharpen),
            ("Box Blur", self.toggle_boxblur),
            ("Gaussian Blur", self.toggle_guassian_blur),
            ("Median", self.toggle_median),
            ("Emboss", self.toggle_emboss),
            ("High Pass", self.toggle_high_pass),
            ("Low Pass", self.toggle_low_pass),
            ("Identity", self.toggle_identity),
        ]

        for idx, (text, command) in enumerate(filters):
            btn = ttk.Button(filter_frame, text=text, command=command)
            btn.grid(row=idx, column=0, pady=5, sticky="ew")

        # Configure the scrollbar to work with the canvas
        canvas.configure(yscrollcommand=scrollbar.set)

        self.image_path = None

        self.video_path = None

        self.image = None

        self.video_capture = None

        self.net = None

        self.classes = None

        self.output_layers = None

        self.filter_mode = None

        self.detect_objects_flag = False

        self.running = False

        self.thread = None

    def select_weights(self):

        self.weights_path = filedialog.askopenfilename()

        self.load_yolo()

    def select_cfg(self):

        self.cfg_path = filedialog.askopenfilename()

        self.load_yolo()

    def select_names(self):

        self.names_path = filedialog.askopenfilename()

        self.load_yolo()

    def load_yolo(self):

        if self.weights_path and self.cfg_path and self.names_path:

            try:

                self.net = cv2.dnn.readNet(self.weights_path, self.cfg_path)

                self.layer_names = self.net.getLayerNames()

                self.output_layers = [
                    self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()
                ]

                with open(self.names_path, "r") as f:

                    self.classes = [line.strip() for line in f.readlines()]

                messagebox.showinfo("YOLO", "YOLO model loaded successfully.")

            except Exception as e:

                messagebox.showerror("YOLO Error", f"Error loading YOLO: {e}")

    def select_image(self):

        self.image_path = filedialog.askopenfilename()

        if self.image_path:
            self.load_image()

    def select_video(self):

        self.video_path = filedialog.askopenfilename()

        if self.video_path:

            if self.running:

                self.stop_running()

            else:

                self.running = True

                self.thread = threading.Thread(target=self.detect_objects_video)

                self.thread.start()

    def toggle_live_video(self):

        if self.running:

            self.stop_running()

        else:

            self.video_capture = cv2.VideoCapture(0)

            self.running = True

            self.thread = threading.Thread(target=self.show_live_video)

            self.thread.start()

    def load_image(self):

        image = cv2.imread(self.image_path)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        image = Image.fromarray(image)

        image = ImageTk.PhotoImage(image)

        self.panel.config(image=image)
        self.panel.image = image

    def toggle_detect_objects(self):

        self.detect_objects_flag = not self.detect_objects_flag

        if self.image_path:

            self.apply_filter_to_image()

    def toggle_edge_detection(self):

        if self.filter_mode == "edge":

            self.filter_mode = None

        else:
            self.filter_mode = "edge"

        if self.image_path:

            self.apply_filter_to_image()

    def toggle_sharpen(self):

        if self.filter_mode == "sharpen":

            self.filter_mode = None

        else:

            self.filter_mode = "sharpen"

        if self.image_path:

            self.apply_filter_to_image()

    def toggle_boxblur(self):

        if self.filter_mode == "boxblur":

            self.filter_mode = None

        else:

            self.filter_mode = "boxblur"

        if self.image_path:

            self.apply_filter_to_image()

    def toggle_guassian_blur(self):

        if self.filter_mode == "guassian":

            self.filter_mode = None

        else:

            self.filter_mode = "guassian"

        if self.image_path:

            self.apply_filter_to_image()

    def toggle_median(self):
        if self.filter_mode == "median":
            self.filter_mode = None
        else:
            self.filter_mode = "median"
        if self.image_path:
            self.apply_filter_to_image()

    def toggle_emboss(self):
        if self.filter_mode == "emboss":
            self.filter_mode = None
        else:
            self.filter_mode = "emboss"
        if self.image_path:
            self.apply_filter_to_image()

    def toggle_high_pass(self):
        if self.filter_mode == "high_pass":
            self.filter_mode = None
        else:
            self.filter_mode = "high_pass"
        if self.image_path:
            self.apply_filter_to_image()

    def toggle_low_pass(self):
        if self.filter_mode == "low_pass":
            self.filter_mode = None
        else:
            self.filter_mode = "low_pass"
        if self.image_path:
            self.apply_filter_to_image()

    def toggle_identity(self):
        if self.filter_mode == "identity":
            self.filter_mode = None
        else:
            self.filter_mode = "identity"
        if self.image_path:
            self.apply_filter_to_image()

    def _detect_objects(self, frame):

        return self.apply_yolo(frame)

    def detect_objects_video(self):

        cap = cv2.VideoCapture(self.video_path)

        while self.running:
            ret, frame = cap.read()

            if not ret:

                break

            if self.detect_objects_flag:

                frame = self.apply_yolo(frame)

            if self.filter_mode == "edge":

                frame = cv2.Canny(frame, 100, 200)

            elif self.filter_mode == "sharpen":

                kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])

                frame = cv2.filter2D(frame, -1, kernel)

            elif self.filter_mode == "boxblur":

                kernel = np.ones((3, 3), np.float32) / 9

                frame = cv2.filter2D(frame, -1, kernel)
            elif self.filter_mode == "identity":
                kernel = np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]])
                frame = cv2.filter2D(frame, -1, kernel)

            elif self.filter_mode == "gaussian_blur":
                kernel = np.array(
                    [
                        [1 / 16, 2 / 16, 1 / 16],
                        [2 / 16, 4 / 16, 2 / 16],
                        [1 / 16, 2 / 16, 1 / 16],
                    ],
                    dtype=np.float32,
                )
                frame = cv2.filter2D(frame, -1, kernel)
            elif self.filter_mode == "median":
                frame = cv2.medianBlur(frame, 3)
            elif self.filter_mode == "emboss":
                kernel = np.array([[-2, -1, 0], [-1, 1, 1], [0, 1, 2]])
                frame = cv2.filter2D(frame, -1, kernel)
            elif self.filter_mode == "high_pass":
                kernel = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])
                frame = cv2.filter2D(frame, -1, kernel)
            elif self.filter_mode == "low_pass":
                kernel = np.array(
                    [
                        [1 / 9, 1 / 9, 1 / 9],
                        [1 / 9, 1 / 9, 1 / 9],
                        [1 / 9, 1 / 9, 1 / 9],
                    ],
                    dtype=np.float32,
                )
                frame = cv2.filter2D(frame, -1, kernel)

            cv2.imshow("Video", frame)

            if cv2.waitKey(1) & 0xFF == ord("q"):

                break
        cap.release()

        cv2.destroyAllWindows()

        self.running = False

    def show_live_video(self):

        while self.running:

            ret, frame = self.video_capture.read()

            if not ret:

                break

            if self.detect_objects_flag:

                frame = self.apply_yolo(frame)

            if self.filter_mode == "edge":

                frame = cv2.Canny(frame, 100, 200)

            elif self.filter_mode == "sharpen":

                kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])

                frame = cv2.filter2D(frame, -1, kernel)
            elif self.filter_mode == "identity":
                kernel = np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]])
                frame = cv2.filter2D(frame, -1, kernel)

            elif self.filter_mode == "boxblur":

                kernel = np.ones((3, 3), np.float32) / 9

                frame = cv2.filter2D(frame, -1, kernel)

            elif self.filter_mode == "gaussian_blur":
                kernel = np.array(
                    [
                        [1 / 16, 2 / 16, 1 / 16],
                        [2 / 16, 4 / 16, 2 / 16],
                        [1 / 16, 2 / 16, 1 / 16],
                    ],
                    dtype=np.float32,
                )
                frame = cv2.filter2D(frame, -1, kernel)
            elif self.filter_mode == "median":
                frame = cv2.medianBlur(frame, 3)

            elif self.filter_mode == "emboss":
                kernel = np.array([[-2, -1, 0], [-1, 1, 1], [0, 1, 2]])
                frame = cv2.filter2D(frame, -1, kernel)
            elif self.filter_mode == "high_pass":
                kernel = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])
                frame = cv2.filter2D(frame, -1, kernel)
            elif self.filter_mode == "low_pass":
                kernel = np.array(
                    [
                        [1 / 9, 1 / 9, 1 / 9],
                        [1 / 9, 1 / 9, 1 / 9],
                        [1 / 9, 1 / 9, 1 / 9],
                    ],
                    dtype=np.float32,
                )
                frame = cv2.filter2D(frame, -1, kernel)
            cv2.imshow("Live Video", frame)

            if cv2.waitKey(1) & 0xFF == ord("q"):

                break

        self.stop_running()

    def stop_running(self):

        self.running = False

        self.detect_objects_flag = False

        self.filter_mode = None

        if self.video_capture is not None:

            self.video_capture.release()

            self.video_capture = None

        cv2.destroyAllWindows()

    def apply_yolo(self, image):

        if not self.net:

            messagebox.showerror(
                "YOLO Error", "YOLO model is not loaded. Please load the model first."
            )

            return image

        height, width, channels = image.shape

        blob = cv2.dnn.blobFromImage(
            image, 0.00392, (416, 416), (0, 0, 0), True, crop=False
        )

        self.net.setInput(blob)

        outs = self.net.forward(self.output_layers)

        class_ids = []

        confidences = []

        boxes = []

        for out in outs:

            for detection in out:

                scores = detection[5:]

                class_id = np.argmax(scores)

                confidence = scores[class_id]

                if confidence > 0.5:

                    center_x = int(detection[0] * width)

                    center_y = int(detection[1] * height)

                    w = int(detection[2] * width)

                    h = int(detection[3] * height)

                    x = int(center_x - w / 2)

                    y = int(center_y - h / 2)

                    if (
                        (x, y, w, h)
                        and isinstance(x, int)
                        and isinstance(y, int)
                        and isinstance(w, int)
                        and isinstance(h, int)
                    ):

                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        for i in range(len(boxes)):

            if i in indexes:

                x, y, w, h = boxes[i]

                label = str(self.classes[class_ids[i]])

                confidence = confidences[i]

                color = (0, 255, 0)

                if (
                    isinstance(x, int)
                    and isinstance(y, int)
                    and isinstance(w, int)
                    and isinstance(h, int)
                ):

                    cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)

                    cv2.putText(
                        image,
                        f"{label} {confidence:.2f}",
                        (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        color,
                        2,
                    )

        return image

    def apply_filter_to_image(self):

        image = cv2.imread(self.image_path)

        if self.detect_objects_flag:

            image = self.apply_yolo(image)

        if self.filter_mode == "edge":

            image = cv2.Canny(image, 100, 200)

        elif self.filter_mode == "sharpen":

            kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])

            image = cv2.filter2D(image, -1, kernel)
        elif self.filter_mode == "identity":
            kernel = np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]])
            image = cv2.filter2D(image, -1, kernel)

        elif self.filter_mode == "boxblur":

            kernel = np.ones((3, 3), np.float32) / 9

            image = cv2.filter2D(image, -1, kernel)
        elif self.filter_mode == "median":
            image = cv2.medianBlur(image, 3)
        elif self.filter_mode == "gaussian_blur":
            kernel = np.array(
                [
                    [1 / 16, 2 / 16, 1 / 16],
                    [2 / 16, 4 / 16, 2 / 16],
                    [1 / 16, 2 / 16, 1 / 16],
                ],
                dtype=np.float32,
            )
            image = cv2.filter2D(image, -1, kernel)
        elif self.filter_mode == "emboss":
            kernel = np.array([[-2, -1, 0], [-1, 1, 1], [0, 1, 2]])
            image = cv2.filter2D(image, -1, kernel)
        elif self.filter_mode == "high_pass":
            kernel = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])
            image = cv2.filter2D(image, -1, kernel)
        elif self.filter_mode == "low_pass":
            kernel = np.array(
                [[1 / 9, 1 / 9, 1 / 9], [1 / 9, 1 / 9, 1 / 9], [1 / 9, 1 / 9, 1 / 9]],
                dtype=np.float32,
            )
            image = cv2.filter2D(image, -1, kernel)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        image = Image.fromarray(image)

        image = ImageTk.PhotoImage(image)

        self.panel.config(image=image)
        self.panel.image = image


if __name__ == "__main__":

    root = tk.Tk()

    app = YOLOFaceDetectionApp(root)
    root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\dell\python\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\dell\AppData\Local\Temp\ipykernel_14668\363716902.py", line 174, in select_weights
    self.load_yolo()
  File "C:\Users\dell\AppData\Local\Temp\ipykernel_14668\363716902.py", line 190, in load_yolo
    if self.weights_path and self.cfg_path and self.names_path:
                             ^^^^^^^^^^^^^
AttributeError: 'YOLOFaceDetectionApp' object has no attribute 'cfg_path'
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\dell\python\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\dell\AppData\Local\Temp\ipykernel_14668\363716902.py", line 180, in select_cfg
    self.load_yolo()
  File "C:\Users\dell\AppData\Local\Temp\ipykernel_14668\363716902.py", line 190, in loa

In [2]:
# Note
# To turn off the live video press the live video button again

NameError: name 'Note' is not defined